In [ ]:
!pip install PyPDF2
!pip install pdfreader


In [ ]:
!pip install chromadb

In [ ]:

#!pip install langchain
!pip install -U langchain-text-splitters

!pip install -U langchain
import langchain
print(langchain.__version__)


1.2.10


In [ ]:
from PyPDF2 import PdfReader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
)



In [ ]:
reader = PdfReader("/content/The_youth_mental_health_crisis_analysis_and_soluti.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]
# Filter the empty strings\
pdf_texts = [text for text in pdf_texts if text]

In [ ]:
#Splitting text and chunking the document ensures that each chunk is token-safe.
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=50
)

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

# Use textwrap.fill to wrap the text
print(f"\nTotal chunks: {len(character_split_texts)}")



Total chunks: 159


In [ ]:
character_split_texts

['The youth mental health crisis:\nanalysis and solutions\nPatrick McGorry1,2*, Hasini Gunasiri1,2*, Cristina Mei1,2,\nSimon Rice1,2and Caroline X. Gao1,2\n1Centre for Youth Mental Health, Faculty of Medicine, Dentistry and Health Sciences, University of\nMelbourne, Parkville, VIC, Australia,2Orygen Youth Health, Parkville, VIC, Australia\nBackground: Since the mid-20th century, mental illness has become a leading\ncause of health burden, particularly among adolescents and emerging adults,\nwith most disorders emerging before the age of 25. Over the past two decades,\nmental ill health has surged to alarming levels, with evidence con ﬁrming that the\nincrease is not just due to better awareness or diagnosis but re ﬂects a genuine\npublic health crisis.\nStudy design/method: We explore the evolving landscape of youth mental\nhealth and its contributing factors, in cluding family dynamics, educational\npressures, climate change, social me dia, and socio-economic challenges,',
 'potential

In [ ]:
#pip install -U sentence-transformers


In [ ]:
#The token-based splitting ensures the chunks fit into our model window.
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)
token_split_texts = []
for text in character_split_texts:
  token_split_texts += token_splitter.split_text(text)
#print(textwrap.fill(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Total chunks: 229


In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Create embedding function
embedding_function = SentenceTransformerEmbeddingFunction()

# Test the embedding function
#print(embedding_function([token_split_texts]))  # should print a vector

# Create Chroma client and collection
chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection(
    "tesla202310k", embedding_function=embedding_function
)


# Add documents
ids = [str(i) for i in range(len(token_split_texts))]
chroma_collection.add(ids=ids, documents=token_split_texts)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
# Confirm count
print(f"Total documents in collection: {chroma_collection.count()}")

Total documents in collection: 229


In [ ]:
!pip install transformers accelerate sentencepiece


In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Pick an open-source instruct model.
# Other good choices: "mistralai/Mistral-7B-Instruct-v0.2" (needs more VRAM),
#                   "Qwen/Qwen2.5-3B-Instruct" (lighter),
#                   "meta-llama/Meta-Llama-3-8B-Instruct" (heavier).
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from google.colab import files
# # Directory to save model and tokenizer locally
# save_directory = "./phi-3-mini-4k-instruct"

# # Save tokenizer and model
# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory )
# !zip -r phi-3-mini-4k-instruct.zip phi-3-mini-4k-instruct

# files.download("phi-3-mini-4k-instruct.zip")


# # ##Use saved model
# # tokenizer = AutoTokenizer.from_pretrained(save_directory)
# # model = AutoModelForCausalLM.from_pretrained(save_directory)

# # input_text = "Explain the importance of AI in healthcare."

# # inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# # # Generate with temperature
# # outputs = model.generate(
# #     **inputs,
# #     max_length=100,
# #     temperature=0.7,    # <-- adjust temperature here
# #     do_sample=True      # Enable sampling to see temperature effect
# # )

# # generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# # print(generated_text)

In [ ]:
# from google.colab import files
# !unzip -q /content/phi-3-mini-4k-instruct.zip -d /content/phi-3-mini-4k-instruct

In [ ]:


SYSTEM_PROMPT = (
    "You are a helpful expert research assistant. Your users are asking questions. "
    "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
    "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic. "
    "Make sure they are complete questions, and that they are related to the original question. "
    "Output one question per line. Do not number the questions."
)

def _clean_questions(text: str, max_q: int = 5):
    lines = []
    for line in text.splitlines():
        line = line.strip()
        # remove bullets/numbers if the model adds them anyway
        line = re.sub(r"^(\-|\*|\d+[\.\)]|\u2022)\s*", "", line).strip()
        if line:
            lines.append(line)
    return lines[:max_q]

def augment_multiple_query(query: str, max_new_tokens: int = 120, temperature: float = 0.7):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]

    # Use chat template if the tokenizer supports it
    if hasattr(tokenizer, "apply_chat_template"):
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        prompt = f"{SYSTEM_PROMPT}\n\nUser: {query}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return _clean_questions(generated, max_q=2)

In [ ]:
# function check
# original_query = "Was there significant turnover in the executive team?"
# augmented_queries = augment_multiple_query(original_query)
# queries = [original_query] + augmented_queries
# queries

In [ ]:
#!huggingface-cli login

In [ ]:
original_query = "Explain Youth mental health trends."
augmented_queries = augment_multiple_query(original_query)

queries = [original_query] + augmented_queries



KeyboardInterrupt: 

In [ ]:
queries

In [ ]:
# retrieve the documents based on the augmented queries
results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents']

for query in augmented_queries:
    print(query)

In [ ]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [ ]:
retrieved_documents

In [ ]:
unique_documents

In [ ]:
#Rerank the retrieved documents based on the relevancy, find top 5 answers for RAG
# select top 5 ranked answers
# Sort the indices of scores in descending order and select the top 3
import numpy as np
from sentence_transformers import CrossEncoder
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
scores = cross_encoder.predict(pairs)
top_indices = np.argsort(scores)[::-1][:5]

# Retrieve the top 5 pairs based on these indices
top_pairs = [pairs[index] for index in top_indices]

# Print the top 5 pairs
print("The top 5 re-ranked answers based on scores are:")
for pair in top_pairs:
    print(pair)

In [ ]:
original_query = """What are The social, economic, and commercial determinants of mental health?"""
augmented_queries = augment_multiple_query(original_query)

queries = [original_query] + augmented_queries

# retrieve the documents based on the augmented queries
results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents']

for query in augmented_queries:
    print(query)

In [ ]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [ ]:
top = sorted(zip(scores, unique_documents), reverse=True)[:5]
for s, doc in top:
    print(s, doc[:160], "...")

In [ ]:
import re
import torch

def summarize_documents_phi(unique_documents, max_new_tokens=200):
    summaries = []
    for i, doc in enumerate(unique_documents):
        messages = [
            {"role": "system", "content": "You are a helpful assistant that summarizes documents accurately and concisely."},
            {"role": "user", "content": f"Summarize the following text in 4-6 bullet points:\n\n{doc}"}
        ]

        # Build chat prompt
        if hasattr(tokenizer, "apply_chat_template"):
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        else:
            prompt = f"{messages[0]['content']}\n\nUser: {doc}\nAssistant:"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                eos_token_id=tokenizer.eos_token_id,
            )

        # Decode only newly generated tokens (exclude prompt tokens)
        gen_ids = output_ids[0][inputs["input_ids"].shape[-1]:]
        summary = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        summaries.append({"doc_index": i, "summary": summary})

    return summaries




In [ ]:
summaries = summarize_documents_phi(unique_documents)


In [ ]:
summaries[0]